In [1]:
import os
import numpy as np

from ehr_ml.clmbr.utils import read_config, read_info

from ehr_ml.extension.timeline import (
    ObservationWithValue,
    TimelineReader,
    Patient,
    PatientDay,
)

In [ ]:
info_dir = "/local-scratch/nigam/projects/jlemmon/cl-clmbr/experiments/main/artifacts/models/clmbr/pretrained/info"
info = read_info(os.path.join(info_dir,'info.json'))

source_file = "/local-scratch/nigam/projects/jlemmon/cl-clmbr/experiments/main/data/extracts/20210723"
timelines = TimelineReader(source_file)

#### Get train data size (num patients, days, codes)

In [ ]:
total_train_patients = len(info['train_patient_ids_with_length'])
total_train_days = np.sum([x[1] for x in info['train_patient_ids_with_length']])
avg_train_days = total_train_days/total_train_patients

total_train_codes, avg_train_codes = 0, 0

for pat_id in [x[0] for x in info['train_patient_ids_with_length']]:
    patient = timelines.get_patient(pat_id)
    num_days = len(patient.days)
    num_codes = 0
    for day in patient.days:
        total_train_codes+=len(day.observations)
        num_codes+=len(day.observations)
        
    avg_train_codes += num_codes/num_days

avg_train_codes/=total_train_patients
    

print(f"\n \
{total_train_patients} total patients\n \
{total_train_days} total days\n \
{avg_train_days} avg days per patient\n \
{total_train_codes} total codes\n \
{avg_train_codes} average codes per day \n\
")

In [ ]:
def obs_with_value_to_str(obs_with_value: ObservationWithValue) -> str:
    return timelines.get_dictionary().get_word(obs_with_value.code)

In [ ]:
vocabs = set()

for pat_id in [x[0] for x in info['train_patient_ids_with_length']]:
    patient = timelines.get_patient(pat_id)
    for i, day in enumerate(patient.days):
        vocabs = vocabs.union(
            set([
                str(timelines.get_dictionary().get_word(a)).split('/')[0]
                for a in day.observations
            ]).union(set([
                obs_with_value_to_str(a).split('/')[0]
                for a in day.observations_with_values
            ]))
        )

In [ ]:
[timelines.get_dictionary().get_word(a) for a in day.observations]